In [204]:
# articleUrl
# articleSeq //불러온 첫번째 seq가 원래꺼보다 클 때
# articleRegDate
# price
# title
# sortDate
# state # 0 1 3

NameError: name 'articleUrl' is not defined

In [9]:
import pandas as pd
import json
import requests
import time
from tqdm import tqdm

한번에 최대 10000개

3일 단위로 계속 불러오기 6개월까지

In [177]:
query="아이폰 13"

url = f'https://search-api.joongna.com/v25/search/product'
param = {
    "filter":{
        "dateFilterParameter":{
            "sortEndDate":"2022-04-13",
            "sortStartDate":"2022-04-12"
        },
        "flawedYn":0,
        "fullPackageYn":0,
        "limitedEditionYn":0,
#         "maxPrice":2000000000,
#         "minPrice":2,
        "tradeType":0
    },
    "productFilter":"ALL",
    "productStates":[
        0,
        1,
        99
    ],
    "searchQuantity":10000,
    "osType":2,
    "searchWord":f'{query}',
    "sort":"RECENT_SORT",
}

In [178]:
#json.dumps
response= requests.post(url,json=(param))
data=response.json()['data']
len(data['items'])

2593

In [182]:
col=['articleUrl',
'articleSeq',
'articleRegDate',
'price',
'title',
'sortDate',
'state'] # 0 1 3]
df=pd.DataFrame(data['items']).drop_duplicates('title')[col]
df=df.dropna()
df.rename(index = {'state': 'is_sold'}, inplace = True)
df['articleSeq']=df['articleSeq'].astype('int')


df

,articleUrl,articleSeq,articleRegDate,price,title,sortDate,state
0,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608326,2022-04-13 23:59:58,2999000,"아이폰13 프로/프로맥스 pro max 당일매입(자급제,당일개통)",2022-04-13 23:59:58,0
1,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608321,2022-04-13 23:59:55,1165000,애플 아이폰13 프로 128g 알파인그린 판매합니다,2022-04-13 23:59:55,0
2,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608275,2022-04-13 23:59:31,4999999,"아이폰13 PRO MAX 프로 맥스 자급제미개봉,당일개통,박스폰삽니다",2022-04-13 23:59:31,0
3,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608191,2022-04-13 23:58:49,1550000,아이폰13 프로 맥스 갤럭시S22 울트라 전기종 삽니다. 자급제 통신사 유심기변,2022-04-13 23:58:49,0
4,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608163,2022-04-13 23:58:32,1000000,"[매입]아이폰13,아이폰전기종 자급제,박스폰,중고폰,파손폰 당일매입",2022-04-13 23:58:32,0
...,...,...,...,...,...,...,...
2587,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910240970,2022-04-12 10:34:19,65000,케이스티파이 아이폰13프로맥스 맥세이프 임팩트 케이스 판매합니다.,2022-04-12 10:34:19,3
2589,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910223599,2022-04-12 09:13:23,1220000,아이폰 13프로 골드 256 애케플 가입팝니다,2022-04-12 09:13:23,3
2590,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910217357,2022-04-12 08:37:22,40000,아이폰13 PRO 프레임바이 에어/에어범퍼 케이스 팝니다.,2022-04-12 08:37:22,3
2591,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910216441,2022-04-12 08:31:42,40000,아이폰13 PRO 케이스티파이 Duty Free 임팩트,2022-04-12 08:31:42,3


In [218]:

detail_list=[]
#
for seq in tqdm(df[df['state']==0]['articleSeq']):
    t=time.time()
    detail_url=f'https://apis.naver.com/cafe-web/cafe-articleapi/v2/cafes/10050146/articles/{seq}'
    response=requests.get(detail_url)
    
    if response.status_code!=200:
#         detail_list.append([seq,"",""])
        continue
    detail=response.json()
    city=detail['result']['articleRegion']['regionName1']
    state=detail['result']['articleRegion']['regionName2']
    img=detail['result']['saleInfo']['imgUrl']
    detail_list.append([seq,city,state])


detail_df=pd.DataFrame(detail_list,columns=["articleSeq",'city','state'])

100%|████████████████████████████████████████████████████████████████████████████████| 370/370 [00:40<00:00,  9.10it/s]


In [226]:
pd.merge(df,detail_df, how='outer',on='articleSeq')

,articleUrl,articleSeq,articleRegDate,price,title,sortDate,state_x,city,state_y
0,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608326,2022-04-13 23:59:58,2999000,"아이폰13 프로/프로맥스 pro max 당일매입(자급제,당일개통)",2022-04-13 23:59:58,0,,
1,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608321,2022-04-13 23:59:55,1165000,애플 아이폰13 프로 128g 알파인그린 판매합니다,2022-04-13 23:59:55,0,NaN,NaN
2,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608275,2022-04-13 23:59:31,4999999,"아이폰13 PRO MAX 프로 맥스 자급제미개봉,당일개통,박스폰삽니다",2022-04-13 23:59:31,0,,
3,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608191,2022-04-13 23:58:49,1550000,아이폰13 프로 맥스 갤럭시S22 울트라 전기종 삽니다. 자급제 통신사 유심기변,2022-04-13 23:58:49,0,,
4,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910608163,2022-04-13 23:58:32,1000000,"[매입]아이폰13,아이폰전기종 자급제,박스폰,중고폰,파손폰 당일매입",2022-04-13 23:58:32,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
425,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910240970,2022-04-12 10:34:19,65000,케이스티파이 아이폰13프로맥스 맥세이프 임팩트 케이스 판매합니다.,2022-04-12 10:34:19,3,NaN,NaN
426,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910223599,2022-04-12 09:13:23,1220000,아이폰 13프로 골드 256 애케플 가입팝니다,2022-04-12 09:13:23,3,NaN,NaN
427,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910217357,2022-04-12 08:37:22,40000,아이폰13 PRO 프레임바이 에어/에어범퍼 케이스 팝니다.,2022-04-12 08:37:22,3,NaN,NaN
428,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910216441,2022-04-12 08:31:42,40000,아이폰13 PRO 케이스티파이 Duty Free 임팩트,2022-04-12 08:31:42,3,NaN,NaN
